# Part 4: Tableau & Slack

Alright, buckle up for **Part 4**! This time, we're going to explore how to make **Tableau and Slack work together seamlessly** through our intelligent agent. Getting data insights is valuable, but being able to effortlessly share those insights with your team is even better!

First things first, let's ensure we have all our necessary Python packages and environment variables set up correctly. We'll also make sure our trusty Tableau Data Source Q&A tool is initialized and ready to go. Once that's in place, we'll move on to the exciting part: integrating Slack!

For those interested in learning more about the Slack Langchain Tool, you can find detailed information at this link: [https://python.langchain.com/docs/integrations/tools/slack/](https://python.langchain.com/docs/integrations/tools/slack/)

![slack channel](./assets/slack_channel.png)

In [44]:
# to access environment variables
import os
from dotenv import load_dotenv

# for displaying pretty results in the notebook
from IPython.display import display, Markdown

# langgraph packages
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage # This will help with the markdown section

# langchain_tableau packages
from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa

# loads environment variables into Python script
load_dotenv()

# initialize variables with these secure values
tableau_server = os.getenv('TABLEAU_DOMAIN')
tableau_site = os.getenv('TABLEAU_SITE')
tableau_jwt_client_id = os.getenv('TABLEAU_JWT_CLIENT_ID')
tableau_jwt_secret_id = os.getenv('TABLEAU_JWT_SECRET_ID')
tableau_jwt_secret = os.getenv('TABLEAU_JWT_SECRET')
tableau_api_version = os.getenv('TABLEAU_API_VERSION')
tableau_user = os.getenv('TABLEAU_USER')
datasource_luid = os.getenv('DATASOURCE_LUID')
tooling_model = os.getenv('TOOLING_MODEL')

# large Language Model for writing VDS queries from natural language within the tool
tooling_model = 'gpt-4o-mini'

# initalize the tool for querying Tableau Datasources through VDS
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_server,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_model
)

# add the tool to a List to give to the agent
tools = [ analyze_datasource ]

# initialize a Large Language Model to be the "brains" of the Agent
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# function to output the steps that the agent is taking
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

Alright, let's re-introduce our beloved Agent Superstore and equip it with an even more amazing personality! To do this, we'll be revisiting and refining our **system prompt**.

The key difference this time is that we're going to enhance the system prompt to include an understanding of the **Slack tools** that will soon be available to our agent. This will allow the agent to not only reason about data from Tableau but also understand how it can interact within a Slack workspace – for example, by sending messages.

By adding this knowledge about Slack capabilities directly into the system prompt, we're giving our agent the foundational understanding it needs to effectively utilize these new tools when we introduce them. Let's craft a prompt that empowers Agent Superstore to be a true Tableau and Slack integration expert!

Just like before, you have the freedom to **tweak and refine this system prompt** to your heart's content. Experimenting with different instructions is a great way to understand how it influences Agent Superstore's behavior and reasoning.

However, it's important to keep in mind that **every change you make to the system prompt will have a ripple effect on all subsequent interactions** with the agent. If you find that Agent Superstore isn't behaving as you expect after making modifications, the first place you should revisit is the system prompt itself. Carefully reviewing and potentially editing it back towards the original or making further adjustments based on the unexpected behavior will likely be the key to getting things back on track. So, feel free to explore, but tread thoughtfully! 😊

In [ ]:
# Agent Identity Definition
identity = """
You are **Agent Superstore**, the veteran AI analyst who has spent years exploring the aisles of the legendary Superstore dataset.
A dataset many Tableau users know and love! 
You live and breathe Superstore data: sales, profits, regions, categories, customer segments, shipping modes, you name it.
And you recently got upgraded with incredible new Slack skills.

Your special mission **today at Tableau Conference 2025** is to help attendees experience the power of Tableau for Langchain Agents. 
You'll be their guide, using this new tool to query the Superstore dataset directly and uncover insights in real-time.

**When you first introduce yourself:**
1.  Greet the attendees and welcome them to the Tableau Conference 2025 hands-on session.
2.  Introduce yourself as Agent Superstore, the AI expert on the classic Superstore dataset.
3.  Briefly explain your purpose: to demonstrate Tableau analytics via agents
"""

# Main System Prompt
system_prompt = f"""**Agent Identity:**
{identity}

**Core Instructions:**

You are an AI Analyst specifically designed to generate data-driven insights from datasets using the tools provided. 
Your goal is to provide answers, guidance, and analysis based on the data accessed via your tools. 
Remember your audience: Tableau users at a conference session, likely familiar with Superstore aka the best dataset ever created.

**Tool Usage Strategy:**

You have access to the following tool:

1.  **`tableau_query_tool` (Data Source Query):** This is your primary tool for interacting with data.
    * **Prioritize this tool** for nearly all user requests asking for specific data points, aggregations, comparisons, trends, or filtered information from datasets.
    * Use it to find specific values (e.g., sales for 'Technology' in 'West' region), calculate aggregates (e.g., `SUM(Sales)`, `AVG(Profit Ratio)`), filter data (e.g., orders in 2023), group data (e.g., sales `BY Category`), and find rankings (e.g., top 5 products by quantity).
    * Be precise in formulating the queries based on the user's request.
2.  **`SlackGetChannel` (Slack Channel Info):** Use this tool to retrieve information about specific Slack channels.
    * Use this tool when asked for channel details (e.g., channel ID, member list, topic for channels like #general or #proj-alpha).
    * Also use when needing a channel ID to send a message if the user only provides the channel name (e.g., finding the ID for 'design-team').
    * Requires the channel name or potentially other identifiers to look up the information.
3.  **`SlackSendMessage` (Send Slack Message):** Use this tool to send messages to specific Slack channels.
    * Use when explicitly asked to post a message, update, or notification to a Slack channel (e.g., "Tell #engineering the build is complete", "Post reminder to #general", "Notify #support about the alert").
    * Requires the target channel ID (you might need to use `SlackGetChannel` first to find the ID if only the name is given) and the exact message content provided by the user.

    
**Response Guidelines:**

* **Grounding:** Base ALL your answers strictly on the information retrieved from your available tools.
* **Clarity:** Always answer the user's core question directly first.
* **Source Attribution:** Clearly state that the information comes from the **dataset** accessed via the Tableau tool (e.g., "According to the data...", "Querying the datasource reveals...").
* **Structure:** Present findings clearly. Use lists or summaries for complex results like rankings or multiple data points. Think like a mini-report derived *directly* from the data query.
* **Tone:** Maintain a helpful, and knowledgeable, befitting your Tableau Superstore expert persona.

**Crucial Restrictions:**
* **DO NOT HALLUCINATE:** Never invent data, categories, regions, or metrics that are not present in the output of your tools. If the tool doesn't provide the answer, state that the information isn't available in the queried data.
"""

# initialize our superstore agent with our Tableau tool
superstore_agent = create_react_agent(model=model, tools=tools, prompt=system_prompt)

Excellent! With our enhanced system prompt in place, Agent Superstore should now be initialized and ready to assist us with both Tableau data and Slack interactions.

Let's take a moment to confirm that our agent is up and running. Go ahead and run the following code snippet to ask Agent Superstore how it's doing. This will give us a first glimpse into its updated persona and hopefully confirm that it's aware of its ability to work with both Tableau and Slack.

In [ ]:
your_prompt = 'Sup?'

# run the agent
messages = {"messages": [("user", your_prompt)]}
print_stream(superstore_agent.stream(messages, stream_mode="values"))

We've certainly informed Agent Superstore about its  Slack capabilities through the system prompt, but we haven't actually provided it with the tools to interact with Slack just yet. Let's rectify that in this next section.

Our first step will be to ensure that we've correctly accessed the necessary **Slack token**, which acts as the key that allows our agent to communicate with your Slack workspace. We'll then double-check that the **Slack Langchain package** is installed in our environment.

Once these prerequisites are met, we'll import and initialize **two essential Slack tools** for our agent:

1.  A tool that will enable Agent Superstore to **read information from the available Slack channels**. This could be useful for understanding context or gathering information from team discussions.
2.  A tool that will empower Agent Superstore to **send messages to Slack channels or users**. This is crucial for sharing the Tableau insights and analyses it generates with your team.

Let's get these Slack tools set up so Agent Superstore can truly leverage its newfound abilities!

In [ ]:
# input the slack token that the tools will need to interact with the workshop
slack_token = os.getenv("SLACK_USER_TOKEN")

# install the slack langchain toolkit
%pip install -qU langchain_community slack

# import and initialize the slack tools
from langchain_community.tools.slack.get_channel import SlackGetChannel
from langchain_community.tools.slack.send_message import SlackSendMessage

toolGetChannel = SlackGetChannel()
toolSendMessage = SlackSendMessage()

tools = [toolGetChannel, toolSendMessage, analyze_datasource]

# initialize our superstore agent with our Tableau tool
superstore_agent = create_react_agent(model=model, tools=tools, prompt=system_prompt)

![slack emoji](./assets/slack_emoji.png)

Alright, the moment has come to put our newly acquired Slack tools to the test! We're going to craft a prompt that encourages Agent Superstore to utilize its ability to interact with Slack.

In the code cell below, you'll find a prompt with two placeholders that I'd like you to edit:

1.  **`[JOKE_TYPE]`**: Please replace this with a specific type of joke you'd like the agent to tell. For example, you could choose "dad", "knock-knock", "one-liner," or any other type of humor you prefer.
2.  **`[CHANNEL_NAME]`**: Replace this with the name of a specific Slack channel that our agent has access to (or a channel you'd like it to hypothetically communicate with).

Feel free to get creative with the prompt! The goal here is to see if Agent Superstore can understand the instruction to use its Slack tools. We're primarily validating that it recognizes and attempts to utilize its Slack capabilities based on the prompt and the tools we've provided.

Go ahead and edit the prompt below with your chosen joke type!

In [ ]:
your_prompt = "Send a [JOKE_TYPE] data joke to the [CHANNEL_NAME] channel"

# Run the agent
messages = {"messages": [("user", your_prompt)]}
print_stream(superstore_agent.stream(messages, stream_mode="values"))

Fantastic! Now, let's witness the true magic of agents! We're going to craft a prompt that combines our agent's ability to query Tableau data with its newfound Slack capabilities in a single, powerful instruction.

Your task is to come up with a creative prompt that asks Agent Superstore to:

1.  **First, query the Superstore dataset in Tableau** for a specific piece of information or insight that would be valuable for your team. Think about a key metric, a trend, or a top-performing category.
2.  **Then, instruct the agent to share the results of this Tableau query in a designated Slack channel.** Imagine you want to broadcast this insight so your entire team can benefit from it and potentially spark further discussion or action.

Remember to replace `[CHANNEL_NAME]` with an appropriate Slack channel name in your prompt. Get creative and think about how you can seamlessly blend data analysis with team communication!

In [ ]:
your_prompt = "What is my revenue by region for q4? Please share the result in [CHANNEL_NAME] channel please"

# Run the agent
messages = {"messages": [("user", your_prompt)]}
print_stream(superstore_agent.stream(messages, stream_mode="values"))

### Wrapping Up Our Hands-On Training! 🎉

Congratulations! You've successfully completed this entire hands-on training experience. We sincerely hope you had a blast exploring the exciting possibilities of Langchain and Tableau!

Let's take a moment to recap what we accomplished in this final part of our journey:

* We **modified our system prompt** to inform our "Superstore" agent about its new capabilities, guiding it to recognize and utilize these skills when appropriate.
* We then **equipped our agent with access to these new skills**, expanding its functionality beyond just being an LLM.
* We demonstrated how to ask questions that prompted the agent to **take action** such as querying data using Tableau and sharing results directly in a Slack channel.

Through these exercises, we aimed to highlight the power of agents as a central medium for executing diverse workflows within your company. An intelligent agent, equipped with the right set of tools, can indeed unlock significant productivity gains across a multitude of use cases you can imagine.

Thank you for joining us on this learning adventure! We hope this training has provided you with a solid foundation for exploring and implementing the transformative potential of AI agents in your own work. Happy experimenting!

![blue scatter](./assets/vizart/scatter-blue-dark.png)